# Combining Neural Networks with Gradient Boosting

## Loading and Transforming Data
We will be working with the CIFAR 10 dataset. A popular dataset for benchmarking image classification performance.

In [8]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import numpy as np

In [9]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize(256),               # Resize to 256x256 pixels
    transforms.CenterCrop(224),           # Crop the image to 224x224 pixels about the center
    transforms.ToTensor(),                # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # Normalize with ImageNet mean
                         std=[0.229, 0.224, 0.225])   # Normalize with ImageNet std
])
# Load the training and test sets
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

# Splitting the training set into training and validation sets
train_size = int(0.8 * len(trainset))  # 80% of the dataset for training
val_size = len(trainset) - train_size  # 20% of the dataset for validation
train_dataset, val_dataset = random_split(trainset, [train_size, val_size])

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

# Create data loaders
trainloader = DataLoader(train_dataset, batch_size=4,
                         shuffle=True, num_workers=2)
valloader = DataLoader(val_dataset, batch_size=4,
                       shuffle=False, num_workers=2)
testloader = DataLoader(testset, batch_size=4,
                        shuffle=False, num_workers=2)

# Classes in CIFAR-10:
# 'plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'


Files already downloaded and verified
Files already downloaded and verified


In [3]:
import torchvision.models as models
import torch.nn as nn

# Load pre-trained ResNet-50
resnet50 = models.resnet50(pretrained=True)

# Remove the last fully connected layer (classification layer)
# to use the model as a feature extractor
modules = list(resnet50.children())[:-1]
resnet50 = nn.Sequential(*modules)

for p in resnet50.parameters():
    p.requires_grad = False  # Freeze the parameters

resnet50.eval()  # Set the model to evaluation mode


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 159MB/s]


Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [10]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

def extract_features(dataloader):
    features_list = []
    labels_list = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            # Move inputs to the device the model is on
            inputs = inputs.to(DEVICE)
            features = resnet50.to(DEVICE)(inputs)

            # Flatten the features to a 1D vector
            features = features.view(features.size(0), -1)

            features_list.append(features.cpu().numpy())
            labels_list.append(labels.cpu().numpy())

    # Concatenate all features and labels into a single NumPy array
    features_array = np.concatenate(features_list, axis=0)
    labels_array = np.concatenate(labels_list, axis=0)

    return features_array, labels_array

# Extract features for train, validation, and test sets
train_features, train_labels = extract_features(trainloader)
val_features, val_labels = extract_features(valloader)
test_features, test_labels = extract_features(testloader)


In [11]:
import xgboost as xgb

# Create DMatrix for XGBoost
dtrain = xgb.DMatrix(train_features, label=train_labels)
dval = xgb.DMatrix(val_features, label=val_labels)
dtest = xgb.DMatrix(test_features, label=test_labels)

# XGBoost parameters
params = {
    'max_depth': 3,
    'eta': 0.1,
    'objective': 'multi:softmax',
    'num_class': 10
}

# Train XGBoost model
bst = xgb.train(params, dtrain, num_boost_round=100, evals=[(dval, 'eval')], early_stopping_rounds=10)

# Save the model
bst.save_model('xgb_model.json')


[0]	eval-mlogloss:2.08032
[1]	eval-mlogloss:1.91896
[2]	eval-mlogloss:1.78940
[3]	eval-mlogloss:1.68011
[4]	eval-mlogloss:1.58597
[5]	eval-mlogloss:1.50446
[6]	eval-mlogloss:1.43190
[7]	eval-mlogloss:1.36728
[8]	eval-mlogloss:1.30894
[9]	eval-mlogloss:1.25662
[10]	eval-mlogloss:1.20796
[11]	eval-mlogloss:1.16333
[12]	eval-mlogloss:1.12275
[13]	eval-mlogloss:1.08542
[14]	eval-mlogloss:1.05093
[15]	eval-mlogloss:1.01879
[16]	eval-mlogloss:0.98890
[17]	eval-mlogloss:0.96061
[18]	eval-mlogloss:0.93534
[19]	eval-mlogloss:0.91109
[20]	eval-mlogloss:0.88886
[21]	eval-mlogloss:0.86780
[22]	eval-mlogloss:0.84811
[23]	eval-mlogloss:0.82900
[24]	eval-mlogloss:0.81147
[25]	eval-mlogloss:0.79498
[26]	eval-mlogloss:0.77901
[27]	eval-mlogloss:0.76443
[28]	eval-mlogloss:0.75073
[29]	eval-mlogloss:0.73726
[30]	eval-mlogloss:0.72444
[31]	eval-mlogloss:0.71260
[32]	eval-mlogloss:0.70121
[33]	eval-mlogloss:0.69047
[34]	eval-mlogloss:0.67994
[35]	eval-mlogloss:0.67045
[36]	eval-mlogloss:0.66120
[37]	eval-m

In [12]:
# Predictions on the test set
preds = bst.predict(dtest)

# Calculate accuracy
accuracy = np.sum(preds == test_labels) / len(test_labels)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 85.41%
